## Signal Processing ENU
August 4, Intern: Diego Varela, Mentor: Surendra Adhikari

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import pandas as pd

In [ ]:
data_folder = '/Users/dlugardo/Desktop/data/ENU_v2' # path to the folder with the data 
locations = ['AASI', 'ASKY', 'AVAN', 'BLAS', 'DANE', 'DGJG', 'SRMP'] # Name of some locations

def get_data(location):
    file_name = str(location) + '.ENU.txt'
    path = os.path.join(data_folder, file_name)

    if os.path.isfile(path):
        data = np.loadtxt(path, skiprows=2)
    else:
        file_name = str(location) + '_ENU.txt'
        path = os.path.join(data_folder, file_name)

        if os.path.isfile(path):
            data = np.loadtxt(path, skiprows=2)
        else:
            raise FileNotFoundError(f"Neither '{location}.ENU.txt' nor '{location}_ENU.txt' found in {data_folder}")
    return data

In [ ]:
np.shape(get_data(locations[0]))

Data file has shape (features, variables). This is how they look like:
 
 | Time (years) | East (mm) | North (mm) | Up (mm) | East_error (mm) | North_error (mm) | Up_error (mm) |
 | ------------ | --------- | ---------- | ------- | --------------- | ---------------- | --------------|
 | 2005.746739  |  81.1990  | -44.5710   | -22.8590|   0.7640        | 1.0650  |  3.8750 |
 | entry 2 ... |   entry 2 ... |entry 2 ... |entry 2 ... |entry 2 ... |entry 2 ... |entry 2 ... |entry 2 ... |


In [ ]:
AASI = get_data('SRMP')
AASI_time = AASI[:,0]
AASI_data = AASI[:,1:4]
AASI_error = AASI[:,4:]

AASI_data_detrended = scipy.signal.detrend(AASI_data, axis = 0)

What I am doing with the data:
1. Plots of the Raw Data
2. Detrended data using scipy:  Average is set to zero. 
3. Fast Fourier Transformations: Through frequency, we can observe how often does the signal repeat (i.e. how long are seasons), 
4. PCA: Understand the main drivers of the signal change.

In [ ]:
years = 2 
fig, axs = plt.subplots(3, 3, figsize=(20, 15))
titles = ['East (mm)', 'North (mm)', 'Up (mm)']

axs[0,0].plot(AASI_time, AASI_data[:,0])
axs[0,1].plot(AASI_time, AASI_data[:,1])
axs[0,2].plot(AASI_time, AASI_data[:,2])

axs[1,0].plot(AASI_time, AASI_data_detrended[:,0])
axs[1,1].plot(AASI_time, AASI_data_detrended[:,1])
axs[1,2].plot(AASI_time, AASI_data_detrended[:,2])

axs[2,0].plot(AASI_time[:365*years], AASI_data_detrended[:365*years,0])
axs[2,1].plot(AASI_time[:365*years], AASI_data_detrended[:365*years,1])
axs[2,2].plot(AASI_time[:365*years], AASI_data_detrended[:365*years,2])

axs[0,0].set_title('East Raw (mm)', weight = 'bold', size = 14)
axs[0,1].set_title('North Raw (mm)', weight = 'bold', size = 14)
axs[0,2].set_title('Up Raw (mm)', weight = 'bold', size = 14)

axs[1,0].set_title('East Detrended (mm)', size = 12)
axs[1,1].set_title('North Detrended (mm)', size = 12)
axs[1,2].set_title('Up Detrended (mm)', size = 12)

fig.suptitle('AASI ENU Data', weight= 'bold', size = 16, y = 0.93)
plt.show()

In [ ]:
#axs[2].scatter(AASI_time, AASI_data[:,2], 0.5, '0.0', lw = 2)
#axs[2].scatter(AASI_time, AASI_data[:,2], 0.51, '1.0', lw = 0,)
#axs[2].scatter(AASI_time, AASI_data[:,2], 0.55, 'C1', lw = 0, alpha = 0.5)

In [ ]:
def moving_average(a, n=30):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

moving_avg_east = moving_average(AASI_data[:,0])
moving_avg_north = moving_average(AASI_data[:,0])
moving_avg_up = moving_average(AASI_data[:,2])

In [ ]:
AASI_error[:,0] + AASI_data[:,0]

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True, figsize=(20,8))
fig.subplots_adjust(hspace=0)

axs[0].fill_between(AASI_time, AASI_data[:,0] + AASI_error[:,0], AASI_data[:,0] - AASI_error[:,0], color = 'g')
axs[0].plot(AASI_time, AASI_data[:,0], marker = '.', ls = '', ms = 1.5, alpha = 0.6)

axs[2].fill_between(AASI_time, AASI_data[:,2] + AASI_error[:,2], AASI_data[:,2] - AASI_error[:,2], color = 'g', label = 'error', alpha = 0.7)
axs[1].plot(AASI_time, AASI_data[:,1], marker = '.', ls = '', ms = 1.5, alpha = 0.6)

axs[2].fill_between(AASI_time, AASI_data[:,2] + AASI_error[:,2], AASI_data[:,2] - AASI_error[:,2], color = 'g', label = 'error', alpha = 0.7)
axs[2].plot(AASI_time, AASI_data[:,2], marker = '.', ls = '', ms = 1.5, alpha = 0.6, label = 'obs')

axs[0].plot(AASI_time[29:], moving_avg_east, marker = '', ls = '-',  alpha = 0.7, c = 'black')
axs[1].plot(AASI_time[29:], moving_avg_north, marker = '', ls = '-',  alpha = 0.7, c = 'black')
axs[2].plot(AASI_time[29:], moving_avg_up, marker = '', ls = '-',  alpha = 0.7, c = 'black', label = 'rolling average, n = 30')

axs[0].set_ylabel('East (mm)')
axs[0].set_xlabel('Date')
axs[1].set_ylabel('North (mm)')
axs[2].set_ylabel('Up (mm)')

axs[2].legend()
fig.suptitle('SRMP Evolution', y = 0.95)


lat/lon/elev (d/d/m):	72.9107	-54.3937	370.6

Through time, we can observe that the sensor is moving to the northwest while moving up. Crust is spreading out.

The detrended versions show 

## Detrended Climatology 
Changed dates from decimals to datetime and made into a pandas dataframe for easier manipulation.

In [ ]:
import datetime

def decimal_year_to_date(decimal_year):
    """
    Converts a decimal year to a datetime.date object.
    """
    year = int(decimal_year)
    fractional_part = decimal_year - year

    # Determine if it's a leap year for accurate day calculation
    is_leap = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
    days_in_year = 366 if is_leap else 365

    # Calculate the number of days from the start of the year
    days_offset = fractional_part * days_in_year

    # Create a datetime object for January 1st of that year
    start_of_year = datetime.date(year, 1, 1)

    # Add the calculated offset in days
    result_date = start_of_year + datetime.timedelta(days=days_offset)

    return result_date

# Example usage:
decimal_year_value = AASI_time[1261]
date_result = decimal_year_to_date(decimal_year_value)
print(f"The decimal year {decimal_year_value} converts to: {date_result}")

In [ ]:
converted_dates = np.array([decimal_year_to_date(dy) for dy in AASI_time])

In [ ]:
df = pd.DataFrame({'date':converted_dates})
df['east'] = pd.Series(AASI_data_detrended[:,0])
df['north'] = pd.Series(AASI_data_detrended[:,1])
df['up'] = pd.Series(AASI_data_detrended[:,2])

df.set_index('date', inplace=True)
df

climatology = df.groupby('date').mean('date').values 
climatology

In [ ]:
df['month_day'] = df.index.map(lambda x: x.strftime('%m-%d'))

climatology = df.groupby('month_day')[['east', 'north', 'up']].mean()
climatology

df['month'] = df.index.map(lambda x: x.strftime('%m'))
climatology_monthly = df.groupby('month')[['east', 'north', 'up']].mean()
climatology_monthly

In [ ]:
df['month'] = df.index.map(lambda x: x.strftime('%m'))
climatology_monthly = df.groupby('month')[['east', 'north', 'up']].mean()
climatology_monthly

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 8))
Time_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
month_ticks = [f'{month:02d}-01' for month in range(1, 13)]

axs[0].plot(climatology['east'])
axs[1].plot(climatology['north'])
axs[2].plot(climatology['up'])

monthly_aligned_index = [f"{month}-15" for month in climatology_monthly.index]

axs[0].plot(monthly_aligned_index, climatology_monthly['east'])
axs[1].plot(monthly_aligned_index, climatology_monthly['north'])
axs[2].plot(monthly_aligned_index, climatology_monthly['up'])

for ax in axs:
    ax.set_xticks(month_ticks) 
    ax.set_xticklabels(Time_months, rotation=45)
    
axs[0].set_title('East Detrended (mm)', size = 12)
axs[1].set_title('North Detrended (mm)', size = 12)
axs[2].set_title('Up Detrended (mm)', size = 12)

fig.suptitle('AASI Climatology', weight= 'bold', size = 16, y = 0.93)
plt.show()

To think about:
- Maybe lets do a 3D Plot to see how the vectors are moving (or at least east and north)
- lets see if its possible to also obtain precipitation and temperature data.
- Up seems to behave like a cosine function.
- Lets calculate the reproducability index for each of them
- pca of day of year vs data. maybe svd too
- we do see that when there is melt (goes down), the sensor also moves to the northeast. Novemeber/December seems to have a small dip (which also appears on the north and east trend) --> check if this also happens on other datasets
- - rolling average?

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.view_init(elev=15, azim=75, roll=0)

ax.plot(climatology_monthly['east'], climatology_monthly['north'], climatology_monthly['up'], lw = 0.5)
ax.set_xlabel("East (mm)")
ax.set_ylabel("North (mm)")
ax.set_zlabel("Up (mm)")

## Spectral Analysis
By doing so, we can break into smaller parts the signals, allowing us to isolate seasonality from variability

In [ ]:
df

In [ ]:
from scipy.fft import dct, idct